In [15]:
import pandas as pd
maisho = pd.read_csv('../data/毎日新聞コーパス/maisho2018-utf8.csv')
maisho

In [16]:
from mojimoji import zen_to_han 
maisho.columns = ['ID', 'C0', 'AD', 'AE', 'AF', 'ZZ', 'T1', 'S1', 'S2', 'T2', 'KB']
maisho.drop(maisho.index[maisho.ZZ == '著作権無'], inplace=True)
maisho = maisho.applymap(lambda x: zen_to_han(str(x), kana=False))
maisho

In [20]:
# ふりがなの削除 ()の中身がひらがなのときのみ
maisho.T2 = maisho.T2.str.replace('\([\u3041-\u309F]+\)', '')
maisho.T2

In [18]:
mai = pd.read_csv('../data/毎日新聞コーパス/mai2018-utf8.csv')
mai

In [19]:
mai.drop(mai.index[mai.ZZ == '著作権無'], inplace=True)
mai.T2 = mai.T2.str.replace('\([\u3041-\u309F]+\)', '')
mai